In [9]:
import open3d as o3d
import numpy as np

# Path to the point cloud file (replace with the correct file path)
file_path = "archive(1)/bildstein_station1_xyz_intensity_rgb/bildstein_station1_xyz_intensity_rgb.txt"

# Load the point cloud (assuming it's a .txt file with XYZ + intensity + RGB)
data = np.loadtxt(file_path)

# Extract XYZ coordinates and RGB values
points = data[:, 0:3]  # First three columns are XYZ coordinates
intensity = data[:, 3]  # Intensity (optional, may skip if not needed)
colors = data[:, 4:7]   # Next three columns are RGB values

# Create Open3D point cloud
#pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(points)
#pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)  # Normalize colors to [0, 1]

# Visualize the point cloud
#o3d.visualization.draw_geometries([pcd])


In [10]:
labels_file_path = "archive(1)/sem8_labels_training/sem8_labels_training/bildstein_station1_xyz_intensity_rgb.labels"
# Load the labels (assuming one label per point)
labels = np.loadtxt(labels_file_path)

# Print first few labels to check
print(labels[:10])

# You can visualize labels by assigning different colors to different classes (optional)
# For now, this just prints them, but further visualization of segmentation can be added later

# Load the labels (assuming one label per point)
labels = np.loadtxt(labels_file_path)

# Print first few labels to check
print(labels[:10])

# You can visualize labels by assigning different colors to different classes (optional)
# For now, this just prints them, but further visualization of segmentation can be added later


[0. 0. 6. 0. 0. 0. 0. 6. 6. 6.]
[0. 0. 6. 0. 0. 0. 0. 6. 6. 6.]


In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class PointCloudDataset(Dataset):
    def __init__(self, point_cloud_file, label_file):
        self.points = np.loadtxt(point_cloud_file)[:, :3]  # XYZ coordinates
        self.labels = np.loadtxt(label_file)  # Corresponding labels for each point

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        point = self.points[idx]
        label = self.labels[idx]
        return torch.tensor(point, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Example usage with one of your point clouds
point_cloud_file = "archive(1)/bildstein_station1_xyz_intensity_rgb/bildstein_station1_xyz_intensity_rgb.txt"
label_file = "archive(1)/sem8_labels_training/sem8_labels_training/bildstein_station1_xyz_intensity_rgb.labels"

# Create the dataset
dataset = PointCloudDataset(point_cloud_file, label_file)

# Create a DataLoader to batch and shuffle the data
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [2]:
import tensorflow as tf
import numpy as np

# Load point cloud (assume you have already loaded it into 'points' and 'labels' arrays)
# Example data loading from earlier
points = np.loadtxt("archive(1)/bildstein_station1_xyz_intensity_rgb/bildstein_station1_xyz_intensity_rgb.txt")[:, 0:3]
labels = np.loadtxt("archive(1)/sem8_labels_training/sem8_labels_training/bildstein_station1_xyz_intensity_rgb.labels")

# Optionally downsample or normalize points
points -= np.mean(points, axis=0)  # Center the point cloud
points /= np.max(np.linalg.norm(points, axis=1))  # Scale to unit sphere

# Reshape and prepare the dataset for TensorFlow (ensure the shape is (num_points, 3) and labels are (num_points,))
print(f"Points shape: {points.shape}, Labels shape: {labels.shape}")

# Shuffle and batch the data
dataset = tf.data.Dataset.from_tensor_slices((points, labels))
dataset = dataset.shuffle(buffer_size=len(points)).batch(32)

2024-10-02 17:19:49.366547: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-02 17:19:49.606475: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 17:19:49.606525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 17:19:49.641228: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 17:19:49.715311: I tensorflow/core/platform/cpu_feature_guar

Points shape: (29697591, 3), Labels shape: (29697591,)


In [15]:
import open3d as o3d
import numpy as np
from scipy.spatial import KDTree

# Ensure 'points' contains the original point cloud data (Reset each time)
original_points = np.copy(points)  # Create a copy of the original points to ensure it's not modified

# Load the original point cloud from the copy
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(original_points)

# Print the number of points in the original point cloud
print(f"Original number of points: {len(original_points)}")

# Apply voxel downsampling
voxel_size = 0.0006  # Adjust voxel size as needed
downsampled_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
downsampled_points = np.asarray(downsampled_pcd.points)

# Print the number of points after downsampling
print(f"Number of points after downsampling: {len(downsampled_points)}")

# Use KDTree to find the nearest original points corresponding to the downsampled points
kdtree = KDTree(original_points)  # Use the original points, not the downsampled
distances, indices = kdtree.query(downsampled_points)

# The "indices" array contains the indices of the original points that correspond to the downsampled points
downsampled_labels = labels[indices]

print(f"Downsampled number of points: {len(downsampled_points)}")


Original number of points: 29697591
Number of points after downsampling: 893679
Downsampled number of points: 893679


In [24]:

def build_pointnet_segmentation_model(num_classes, input_shape=(None, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    # Add more layers and dropout to improve generalization
    x = layers.Conv1D(128, kernel_size=1, activation='relu')(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv1D(256, kernel_size=1, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv1D(512, kernel_size=1, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    global_features = layers.GlobalMaxPooling1D()(x)

    num_points = tf.shape(inputs)[1]
    global_features_repeated = tf.tile(tf.expand_dims(global_features, axis=1), [1, num_points, 1])

    x = layers.Concatenate()([inputs, global_features_repeated])

    x = layers.Conv1D(256, kernel_size=1, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv1D(128, kernel_size=1, activation='relu')(x)
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

In [17]:
import tensorflow as tf

# Add a batch dimension to your downsampled point cloud and labels
downsampled_points = np.expand_dims(downsampled_points, axis=0)  # Shape (1, num_points, 3)
downsampled_labels = np.expand_dims(downsampled_labels, axis=0)  # Shape (1, num_points)

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((downsampled_points, downsampled_labels)).batch(1)

In [31]:
from tensorflow.keras import layers, models

# Define the PointNet++ model
def build_pointnet_segmentation_model(num_classes, input_shape=(None, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    # PointNet++ architecture (simplified for segmentation)
    x = layers.Conv1D(64, kernel_size=1, activation='relu')(inputs)
    x = layers.Conv1D(128, kernel_size=1, activation='relu')(x)
    x = layers.Conv1D(256, kernel_size=1, activation='relu')(x)
    global_features = layers.GlobalMaxPooling1D()(x)

    # Use tf.tile to repeat the global features dynamically for each point
    num_points = tf.shape(inputs)[1]
    global_features_repeated = tf.tile(tf.expand_dims(global_features, axis=1), [1, num_points, 1])

    # Concatenate global features with point features
    x = layers.Concatenate()([inputs, global_features_repeated])

    # Final point-wise segmentation layers
    x = layers.Conv1D(128, kernel_size=1, activation='relu')(x)
    x = layers.Conv1D(64, kernel_size=1, activation='relu')(x)
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Define the number of classes based on your dataset
num_classes = len(np.unique(downsampled_labels))  # Adjust for your dataset
model = build_pointnet_segmentation_model(num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to verify
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, None, 3)]            0         []                            
                                                                                                  
 conv1d_36 (Conv1D)          (None, None, 64)             256       ['input_7[0][0]']             
                                                                                                  
 conv1d_37 (Conv1D)          (None, None, 128)            8320      ['conv1d_36[0][0]']           
                                                                                                  
 conv1d_38 (Conv1D)          (None, None, 256)            33024     ['conv1d_37[0][0]']           
                                                                                            

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)  # Reduce learning rate
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [32]:
from sklearn.utils import class_weight

# Compute class weights to handle class imbalance
class_weights = class_weight.compute_class_weight('balanced', np.unique(downsampled_labels), downsampled_labels.flatten())
class_weights_dict = dict(enumerate(class_weights))

# Add class weights during training
model.fit(dataset, epochs=20, class_weight=class_weights_dict)


TypeError: too many positional arguments

In [30]:
# Train the PointNet++ segmentation model
epochs = 20  # Adjust based on your needs
batch_size = 32  # Adjust based on your hardware capabilities
model.fit(dataset, epochs=epochs, batch_size=batch_size)

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 1.2339 - accuracy: 0.7213
Epoch 2/20
1/1 [==============================] - 3s 3s/step - loss: 2.3638 - accuracy: 0.0928
Epoch 3/20
1/1 [==============================] - 3s 3s/step - loss: 1.5470 - accuracy: 0.7213
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 4.3325 - accuracy: 0.7213
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 1.1686 - accuracy: 0.7213
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 1.5548 - accuracy: 0.7160
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 1.4773 - accuracy: 0.7213
Epoch 8/20
1/1 [==============================] - 3s 3s/step - loss: 1.3183 - accuracy: 0.7213
Epoch 9/20
1/1 [==============================] - 3s 3s/step - loss: 1.0887 - accuracy: 0.7213
Epoch 10/20
1/1 [==============================] - 3s 3s/step - loss: 1.1768 - accuracy: 0.7213
Epoch 11/20
1/1 [==============================] 

KeyboardInterrupt: 

In [21]:
# Predict segmentation labels for the point cloud
predicted_labels = model.predict(downsampled_points)  # Predict for the downsampled points

# Convert predicted probabilities to class labels
predicted_labels = np.argmax(predicted_labels, axis=-1).flatten()  # Shape: (num_points,)

# Print the first few predicted labels to verify
print(predicted_labels[:10])

1/1 [==============================] - 1s 1s/step
[0 0 0 0 0 0 0 0 0 0]


In [22]:
import open3d as o3d

# Create an Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(downsampled_points[0])  # Use the downsampled points

# Assign random colors to each class label
num_classes = len(np.unique(predicted_labels))
colors = np.random.rand(num_classes, 3)  # Generate random colors for each class
pcd.colors = o3d.utility.Vector3dVector(colors[predicted_labels])

# Visualize the segmented point cloud
o3d.visualization.draw_geometries([pcd])